## Deliverable 2. Create a Customer Travel Destinations Map.

In [28]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import matplotlib.pyplot as plt
import os
import json
import gmaps.datasets
import ipywidgets as widgets



# Import API key
from config import google_api_key

# Configure gmaps API key
gmaps.configure(api_key=google_api_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,Clyde River,70.4692,-68.5914,overcast clouds,19.63,86,100,13.80,CA
1,1,Avarua,-21.2078,-159.7750,broken clouds,80.65,83,75,5.75,CK
2,2,Saint-Pierre,-21.3393,55.4781,clear sky,73.90,49,0,4.61,RE
3,3,Arraial Do Cabo,-22.9661,-42.0278,clear sky,69.80,76,5,14.52,BR
4,4,Hamilton,39.1834,-84.5333,scattered clouds,60.24,63,40,10.36,US


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
user_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                  (city_data_df["Max Temp"] >= min_temp)]

user_cities_df.head(10)

,City_ID,City,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country
1,1,Avarua,-21.2078,-159.7750,broken clouds,80.65,83,75,5.75,CK
2,2,Saint-Pierre,-21.3393,55.4781,clear sky,73.90,49,0,4.61,RE
3,3,Arraial Do Cabo,-22.9661,-42.0278,clear sky,69.80,76,5,14.52,BR
5,5,Rikitea,-23.1203,-134.9692,broken clouds,76.15,75,62,15.95,PF
9,9,Butaritari,3.0707,172.7902,light rain,82.40,79,87,14.81,KI
12,12,Kapaa,22.0752,-159.3190,broken clouds,80.58,83,75,13.80,US
19,19,Ulaanbaatar,47.9077,106.8832,broken clouds,77.13,14,75,8.95,MN
20,20,Zhob,31.3411,69.4481,clear sky,79.81,5,0,17.22,PK
24,24,Kavieng,-2.5744,150.7967,scattered clouds,85.64,61,45,14.43,PG
28,28,Charters Towers,-20.1000,146.2667,scattered clouds,76.05,55,36,5.73,AU


In [9]:
# 4a. Determine if there are any empty rows.
user_cities_df.isnull().sum()


City_ID                0
City                   0
Lat                    0
Lng                    0
Current Description    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
dtype: int64

In [12]:
print(len(user_cities_df))
user_cities_df.count()

694


City_ID                694
City                   694
Lat                    694
Lng                    694
Current Description    694
Max Temp               694
Humidity               694
Cloudiness             694
Wind Speed             694
Country                694
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = user_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

694


,City_ID,City,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country
1,1,Avarua,-21.2078,-159.7750,broken clouds,80.65,83,75,5.75,CK
2,2,Saint-Pierre,-21.3393,55.4781,clear sky,73.90,49,0,4.61,RE
3,3,Arraial Do Cabo,-22.9661,-42.0278,clear sky,69.80,76,5,14.52,BR
5,5,Rikitea,-23.1203,-134.9692,broken clouds,76.15,75,62,15.95,PF
9,9,Butaritari,3.0707,172.7902,light rain,82.40,79,87,14.81,KI


In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Avarua,CK,80.65,broken clouds,-21.2078,-159.7750,
2,Saint-Pierre,RE,73.90,clear sky,-21.3393,55.4781,
3,Arraial Do Cabo,BR,69.80,clear sky,-22.9661,-42.0278,
5,Rikitea,PF,76.15,broken clouds,-23.1203,-134.9692,
9,Butaritari,KI,82.40,light rain,3.0707,172.7902,
12,Kapaa,US,80.58,broken clouds,22.0752,-159.3190,
19,Ulaanbaatar,MN,77.13,broken clouds,47.9077,106.8832,
20,Zhob,PK,79.81,clear sky,31.3411,69.4481,
24,Kavieng,PG,85.64,scattered clouds,-2.5744,150.7967,
28,Charters Towers,AU,76.05,scattered clouds,-20.1000,146.2667,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [16]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Avarua,CK,80.65,broken clouds,-21.2078,-159.7750,Paradise Inn
2,Saint-Pierre,RE,73.90,clear sky,-21.3393,55.4781,Alize Plage
3,Arraial Do Cabo,BR,69.80,clear sky,-22.9661,-42.0278,Pousada Porto Praia
5,Rikitea,PF,76.15,broken clouds,-23.1203,-134.9692,People ThankYou
9,Butaritari,KI,82.40,light rain,3.0707,172.7902,Isles Sunset Lodge
12,Kapaa,US,80.58,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
19,Ulaanbaatar,MN,77.13,broken clouds,47.9077,106.8832,The Blue Sky Hotel and Tower
20,Zhob,PK,79.81,clear sky,31.3411,69.4481,ROOM # 105 DUBAI HOTEL ZHOB
24,Kavieng,PG,85.64,scattered clouds,-2.5744,150.7967,Nusa Island Retreat
28,Charters Towers,AU,76.05,scattered clouds,-20.1000,146.2667,Cattleman's Rest Motor Inn


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
print(fig)

Figure(layout=FigureLayout(height='420px'))
